In [1]:
import pandas
import os

In [2]:
root_path = 'data/kaggle_2018/DataFiles'

teams = pandas.read_csv(os.path.join(root_path, 'Teams.csv'))
slots = pandas.read_csv(os.path.join(root_path, 'NCAATourneySlots.csv'))
seeds = pandas.read_csv(os.path.join(root_path, 'NCAATourneySeeds.csv'))
results = pandas.read_csv(os.path.join(root_path, 'NCAATourneyCompactResults.csv'))

In [3]:
results['minID'] = results[['WTeamID', 'LTeamID']].min(axis=1)
results['maxID'] = results[['WTeamID', 'LTeamID']].max(axis=1)

In [5]:
df = slots.merge(seeds, left_on=['Season', 'StrongSeed'], right_on=['Season', 'Seed'], how='left')
df = df.rename(index=str, columns={"TeamID": "StrongTeamID"}).drop('Seed', axis=1)
df = df.merge(seeds, left_on=['Season', 'WeakSeed'], right_on=['Season', 'Seed'], how='left')
df = df.rename(index=str, columns={"TeamID": "WeakTeamID"}).drop('Seed', axis=1)

df[df['Season'] == 2015][:5]

,Season,Slot,StrongSeed,WeakSeed,StrongTeamID,WeakTeamID
1916,2015,W11,W11a,W11b,1129.0,1173.0
1917,2015,X16,X16a,X16b,1316.0,1352.0
1918,2015,Y16,Y16a,Y16b,1214.0,1264.0
1919,2015,Z11,Z11a,Z11b,1140.0,1279.0
1920,2015,R1W1,W01,W16,1437.0,1248.0


In [9]:
team_pts_df = None

for season in range(2010, 2017+1):
    # initialize
    teams_68 = list(seeds[(seeds['Season'] == season)]['TeamID'])

    # play-in games
    pi_games = ~(df['Slot'].str.contains('R')) & (df['Season'] == season)
    pi = df[pi_games]

    pi.loc[pi_games, 'minID'] = pi[['StrongTeamID', 'WeakTeamID']].min(axis=1)
    pi.loc[pi_games, 'maxID'] = pi[['StrongTeamID', 'WeakTeamID']].max(axis=1)

    pi = pi.merge(results, on=['Season', 'minID', 'maxID'], how='left')

    pi = pi.merge(teams[['TeamID', 'TeamName']], left_on=['StrongTeamID'], right_on=['TeamID'], how='left')
    pi = pi.rename(index=str, columns={"TeamName": "StrongTeamName"}).drop('TeamID', axis=1)
    pi = pi.merge(teams[['TeamID', 'TeamName']], left_on=['WeakTeamID'], right_on=['TeamID'], how='left')
    pi = pi.rename(index=str, columns={"TeamName": "WeakTeamName"}).drop('TeamID', axis=1)
    pi = pi.merge(teams[['TeamID', 'TeamName']], left_on=['WTeamID'], right_on=['TeamID'], how='left')
    pi = pi.rename(index=str, columns={"TeamName": "WTeamName"}).drop('TeamID', axis=1)
    pi = pi.merge(teams[['TeamID', 'TeamName']], left_on=['LTeamID'], right_on=['TeamID'], how='left')
    pi = pi.rename(index=str, columns={"TeamName": "LTeamName"}).drop('TeamID', axis=1)

    # regular rounds
    rounds = [pi]

    for rnd in range(1, 6 + 1):
        last_rnd = rounds[-1]

        r_games = (df['Slot'].str.contains('R{}..'.format(rnd))) & (df['Season'] == season)
        r = df[r_games]

        r = r.merge(last_rnd[['Slot', 'WTeamID']], left_on='StrongSeed', right_on='Slot', how='left', suffixes=['', '__'])
        r.loc[r['StrongTeamID'].isnull(), 'StrongTeamID'] = r['WTeamID']
        r = r.drop(['Slot__', 'WTeamID'], axis=1)

        r = r.merge(last_rnd[['Slot', 'WTeamID']], left_on='WeakSeed', right_on='Slot', how='left', suffixes=['', '__'])
        r.loc[r['WeakTeamID'].isnull(), 'WeakTeamID'] = r['WTeamID']
        r = r.drop(['Slot__', 'WTeamID'], axis=1)

        r['minID'] = r[['StrongTeamID', 'WeakTeamID']].min(axis=1)
        r['maxID'] = r[['StrongTeamID', 'WeakTeamID']].max(axis=1)

        r = r.merge(results, on=['Season', 'minID', 'maxID'], how='left')

        r = r.merge(teams[['TeamID', 'TeamName']], left_on=['StrongTeamID'], right_on=['TeamID'], how='left')
        r = r.rename(index=str, columns={"TeamName": "StrongTeamName"}).drop('TeamID', axis=1)
        r = r.merge(teams[['TeamID', 'TeamName']], left_on=['WeakTeamID'], right_on=['TeamID'], how='left')
        r = r.rename(index=str, columns={"TeamName": "WeakTeamName"}).drop('TeamID', axis=1)
        r = r.merge(teams[['TeamID', 'TeamName']], left_on=['WTeamID'], right_on=['TeamID'], how='left')
        r = r.rename(index=str, columns={"TeamName": "WTeamName"}).drop('TeamID', axis=1)
        r = r.merge(teams[['TeamID', 'TeamName']], left_on=['LTeamID'], right_on=['TeamID'], how='left')
        r = r.rename(index=str, columns={"TeamName": "LTeamName"}).drop('TeamID', axis=1)

        rounds.append(r.copy())

    team_pts = dict([(t, 0) for t in teams_68])

    weights = {1: 1,
               2: 2,
               3: 4,
               4: 8,
               5: 16,
               6: 32}

    for rnd in range(1, 6+1):
        for t in list(rounds[rnd]['WTeamID']):
            team_pts[t] += weights[rnd]

    teamID, pts = zip(*team_pts.iteritems())
    if team_pts_df is None:
        team_pts_df = pandas.DataFrame({'TeamID': teamID, 'Points': pts, 'Season': [season]*len(teamID)}).merge(teams[['TeamID', 'TeamName']], on='TeamID')
    else:
        team_pts_df = team_pts_df.append(pandas.DataFrame({'TeamID': teamID, 'Points': pts, 'Season': [season]*len(teamID)}).merge(teams[['TeamID', 'TeamName']], on='TeamID'))
        
team_pts_df[:5]

,Points,Season,TeamID,TeamName
0,1,2010,1281,Missouri
1,0,2010,1155,Clemson
2,0,2010,1285,Montana
3,0,2010,1288,Morgan St
4,0,2010,1196,Florida


In [12]:
team_pts_df[team_pts_df['Season'] == 2017]

,Points,Season,TeamID,TeamName
0,1,2017,1153,Cincinnati
1,0,2017,1195,FL Gulf Coast
2,0,2017,1413,UC Davis
3,7,2017,1196,Florida
4,0,2017,1291,Mt St Mary's
5,1,2017,1292,MTSU
6,0,2017,1166,Creighton
7,0,2017,1423,UNC Wilmington
8,1,2017,1425,USC
9,0,2017,1411,TX Southern


In [10]:
output_file = 'team_points.csv'
team_pts_df.to_csv(output_file, sep='|')